In [1]:
import whisperx
import torch
from docx import Document
import os

# Çalışma dizinini tangible klasörüne ayarla
os.chdir('..')

# Şu anki çalışma dizinini kontrol et
print(os.getcwd())


c:\Users\musta\Documents\Python Projects\tangible\venv_whisperx\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [allow_tf32, disable_jit_profiling]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []


c:\Users\musta\Documents\Python Projects\tangible


In [ ]:
# Cihazı ve uygun compute_type'ı belirle
device = "cuda" if torch.cuda.is_available() else "cpu"
compute_type = "float16" if device == "cuda" else "int8"
print(f"Cihaz: {device}, Compute Type: {compute_type}")

# WhisperX modelini yükle
print("WhisperX modeli yükleniyor...")
model = whisperx.load_model("large-v2", device, compute_type=compute_type)
print("WhisperX modeli yüklendi.")

# 'data' klasöründeki ses dosyasının yolunu belirtin
audio_file = os.path.join("data", "Yönetici_1 Erdem Erdoğdu2.wav")
print(f"Ses dosyası: {audio_file}")

# Ses dosyasını transkribe et
print("Ses dosyası transkribe ediliyor...")
result = model.transcribe(audio_file, language='tr')
print("Transkripsiyon tamamlandı.")

# Zaman hizalaması için model ve metadata yükle
print("Zaman hizalama modeli yükleniyor...")
model_a, metadata = whisperx.load_align_model(language_code='tr', device=device)
print("Zaman hizalama modeli yüklendi.")

# Zaman hizalaması yap
print("Zaman hizalaması yapılıyor...")
result_aligned = whisperx.align(result["segments"], model_a, metadata, audio_file, device)
print("Zaman hizalaması tamamlandı.")

# Konuşmacı ayrımı için pyannote.audio tabanlı modeli yükle
print("Konuşmacı ayrımı modeli yükleniyor...")
diarize_model = whisperx.DiarizationPipeline(use_auth_token="YOUR_HUGGINGFACE_TOKEN", device=device)
print("Konuşmacı ayrımı modeli yüklendi.")

# Konuşmacı ayrımı sonuçlarını al
print("Konuşmacı ayrımı yapılıyor...")
diarize_segments = diarize_model(audio_file)
print("Konuşmacı ayrımı tamamlandı.")

# Konuşmacı bilgilerini transkripsiyonla birleştir
print("Konuşmacı bilgileri transkripsiyonla birleştiriliyor...")
result_segments, word_segments = whisperx.assign_word_speakers(diarize_segments, result_aligned)
print("Birleştirme tamamlandı.")

# Sonuçları bir Word belgesine kaydet
print("Sonuçlar Word belgesine kaydediliyor...")
doc = Document()
doc.add_heading('Transkripsiyon Sonuçları', level=1)

for segment in result_segments:
    speaker = segment['speaker']
    text = segment['text']
    doc.add_paragraph(f"{speaker}: {text}")

# Ses dosyasının adını al ve uzantısını '.docx' olarak değiştir
audio_filename = os.path.splitext(os.path.basename(audio_file))[0]
output_filename = f"{audio_filename}.docx"

# Word belgesini kaydet
output_path = os.path.join("data", output_filename)
doc.save(output_path)
print(f"Sonuçlar '{output_path}' dosyasına kaydedildi.")


In [ ]:
import whisperx
import torch
from docx import Document
import os
from tqdm import tqdm
import time  # Simülasyon amacıyla eklendi
from dotenv import load_dotenv

# .env dosyasını yükle
load_dotenv()
# HF Token'ı ENV'den al
hf_token = os.getenv("HF_TOKEN")

# Toplam adım sayısı
total_steps = 10

# İlerleme çubuğunu başlat
pbar = tqdm(total=total_steps, desc="İşlem Adımları", unit="adım")

# 1. Adım: Cihazı ve uygun compute_type'ı belirle
device = "cuda" if torch.cuda.is_available() else "cpu"
compute_type = "float16" if device == "cuda" else "int8"
print(f"Cihaz: {device}, Compute Type: {compute_type}")
pbar.update(1)

# 2. Adım: WhisperX modelini yükle
print("WhisperX modeli yükleniyor...")
model = whisperx.load_model("large-v2", device, compute_type=compute_type)
print("WhisperX modeli yüklendi.")
pbar.update(1)

# 3. Adım: 'data' klasöründeki ses dosyasının yolunu belirtin
audio_file = os.path.join("data", "Yönetici_1 Erdem Erdoğdu2.wav")
print(f"Ses dosyası: {audio_file}")
pbar.update(1)

# 4. Adım: Ses dosyasını transkribe et
print("Ses dosyası transkribe ediliyor...")
result = model.transcribe(audio_file, language='tr')
print("Transkripsiyon tamamlandı.")
pbar.update(1)

# 5. Adım: Zaman hizalama modeli ve metadata yükle
print("Zaman hizalama modeli yükleniyor...")
model_a, metadata = whisperx.load_align_model(language_code='tr', device=device)
print("Zaman hizalama modeli yüklendi.")
pbar.update(1)

# 6. Adım: Zaman hizalaması yap
print("Zaman hizalaması yapılıyor...")
result_aligned = whisperx.align(result["segments"], model_a, metadata, audio_file, device)
print("Zaman hizalaması tamamlandı.")
pbar.update(1)

# 7. Adım: Konuşmacı ayrımı modeli yükleniyor
print("Konuşmacı ayrımı modeli yükleniyor...")
diarize_model = whisperx.DiarizationPipeline(use_auth_token=hf_token, device=device)
print("Konuşmacı ayrımı modeli yüklendi.")
pbar.update(1)

# 8. Adım: Konuşmacı ayrımı yapılıyor
print("Konuşmacı ayrımı yapılıyor...")
diarize_segments = diarize_model(audio_file)
print("Konuşmacı ayrımı tamamlandı.")
pbar.update(1)

# 9. Adım: Konuşmacı bilgileri transkripsiyonla birleştiriliyor
print("Konuşmacı bilgileri transkripsiyonla birleştiriliyor...")
result_segments, word_segments = whisperx.assign_word_speakers(diarize_segments, result_aligned)
print("Birleştirme tamamlandı.")
pbar.update(1)

# 10. Adım: Sonuçlar Word belgesine kaydediliyor
"""
print("Sonuçlar Word belgesine kaydediliyor...")
doc = Document()
doc.add_heading('Transkripsiyon Sonuçları', level=1)

for segment in result_segments:
    speaker = segment['speaker']
    text = segment['text']
    doc.add_paragraph(f"{speaker}: {text}")

# Ses dosyasının adını al ve uzantısını '.docx' olarak değiştir
audio_filename = os.path.splitext(os.path.basename(audio_file))[0]
output_filename = f"{audio_filename}.docx"

# Word belgesini kaydet
output_path = os.path.join("data", output_filename)
doc.save(output_path)
print(f"Sonuçlar '{output_path}' dosyasına kaydedildi.")
pbar.update(1)

# İlerleme çubuğunu tamamla
pbar.close()
"""


In [ ]:
from docx import Document

# Word belgesi oluştur
doc = Document()

# Birleştirilmiş metinleri tutacak değişkenler
merged_transcriptions = []
current_speaker = None
current_text = ""

# Tüm segmentlerde dolaş
for segment in aligned_segments:
    speaker = segment.get('speaker', 'Bilinmeyen Konuşmacı')
    text = segment['text']

    # Eğer konuşmacı değişirse veya ilk segment ise
    if current_speaker != speaker:
        if current_speaker is not None:  # Önceki konuşmacının metnini kaydet
            merged_transcriptions.append(f"{current_speaker}: {current_text}")
        # Yeni konuşmacı için değişkenleri sıfırla
        current_speaker = speaker
        current_text = text
    else:
        # Aynı konuşmacı ise metni birleştir
        current_text += f" {text}"

# Son konuşmacının metnini ekle
if current_speaker is not None:
    merged_transcriptions.append(f"{current_speaker}: \n{current_text}")

# Word belgesine yaz
for transcription in merged_transcriptions:
    doc.add_paragraph(transcription)

# Belgeyi kaydet
doc.save("birleştirilmiş_transkripsiyon.docx")
print("Birleştirilmiş transkripsiyon kaydedildi.")
print(merged_transcriptions)


In [2]:
# WhisperX 
import whisperx
import torch
from docx import Document
import os
from tqdm import tqdm
import time
from dotenv import load_dotenv

# .env dosyasını yükle

load_dotenv()
# HF Token'ı ENV'den al
hf_token = os.getenv("HF_TOKEN")

# Toplam adım sayısı
total_steps = 10

# İlerleme çubuğunu başlat
pbar = tqdm(total=total_steps, desc="Process Step", unit="step")

# 1. Cihazı ve uygun compute_type'ı belirle
device = "cuda" if torch.cuda.is_available() else "cpu"
compute_type = "float16" if device == "cuda" else "int8"
print(f"Device: {device}, Compute Type: {compute_type}")
pbar.update(1)

# 2. WhisperX modelini yükle
print("Loading WhisperX model...")
model = whisperx.load_model("large-v2", device, compute_type=compute_type)
print("WhisperX model loaded.")
pbar.update(1)

# 3. 'data' klasöründeki ses dosyasının yolunu belirtin
audio_file = os.path.join("data", "Görüşme10 Selma Zengin.m4a")
print(f"Audio file: {audio_file}")
pbar.update(1)

# 4. Ses dosyasını transkribe et
print("Transcribing audio file...")
result = model.transcribe(audio_file, language='tr')
print("Transcription completed.")
pbar.update(1)

# 5. Zaman hizalama modeli ve metadata yükle
print("Loading time alignment model...")
model_a, metadata = whisperx.load_align_model(language_code='tr', device=device)
print("Time alignment model loaded.")
pbar.update(1)

# 6. Zaman hizalaması yap
print("Processing time alignment...")
result_aligned = whisperx.align(result["segments"], model_a, metadata, audio_file, device)
print("Time alignment completed.")
pbar.update(1)

# 7. Konuşmacı ayrımı modeli yükleniyor
print("Loading speaker diarization model...")
diarize_model = whisperx.DiarizationPipeline(use_auth_token=hf_token, device=device)
print("Speaker diarization model loaded.")
pbar.update(1)

# 8. Konuşmacı ayrımı yapılıyor
print("Processing speaker diarization...")
diarize_segments = diarize_model(audio_file)
print("Speaker diarization completed.")
pbar.update(1)

# 9. Konuşmacı bilgileri transkripsiyonla birleştiriliyor
print("Speaker information is being merged with the transcription...")
#result_segments, word_segments = whisperx.assign_word_speakers(diarize_segments, result_aligned)

# `result_aligned['segments']` ve `diarize_segments` manuel olarak birleştiriliyor
aligned_segments = result_aligned['segments']  # Metin segmentleri
diarized_segments = []  # Birleştirilmiş sonuçları saklamak için liste

for segment in aligned_segments:
    segment_start = segment['start']
    segment_end = segment['end']
    text = segment['text']

    # Konuşmacıyı zaman aralığına göre eşleştir
    speaker = "Unknown Speaker"  # Varsayılan
    for diarize in diarize_segments.itertuples():
        if diarize.start <= segment_start <= diarize.end or diarize.start <= segment_end <= diarize.end:
            speaker = diarize.speaker
            break

    # Birleştirilmiş sonucu ekle
    diarized_segments.append({'speaker': speaker, 'text': text})

# Birleştirilmiş sonuçları işlemek için değişkenler
merged_transcriptions = []
current_speaker = None
current_text = ""

# Birleştirilmiş segmentlerde dolaş ve konuşmacı metinlerini birleştir
for segment in diarized_segments:
    speaker = segment['speaker']
    text = segment['text']

    # Eğer konuşmacı değişirse veya ilk segment ise
    if current_speaker != speaker:
        if current_speaker is not None:  # Önceki konuşmacının metnini kaydet
            merged_transcriptions.append((current_speaker, current_text))
        current_speaker = speaker  # Yeni konuşmacıya geç
        current_text = text  # Yeni metni başlat
    else:
        # Aynı konuşmacı ise metni birleştir
        current_text += f" {text}"

# Son konuşmacının metnini ekle
if current_speaker is not None:
    merged_transcriptions.append((current_speaker, current_text))

print("Merging completed.")
pbar.update(1)

# 10. Adım: Sonuçlar Word belgesine kaydediliyor
print("Saving results to the Word document...")
audio_filename = os.path.splitext(os.path.basename(audio_file))[0]
doc = Document()
doc.add_heading(f'Transkripsiyon Sonuçları: {audio_filename}', level=1)

# Her konuşmacıyı ve metni ayrı paragraflara yaz
for speaker, text in merged_transcriptions:
    doc.add_paragraph(f"{speaker}:")
    doc.add_paragraph(text)

# Word dosyasını kaydet
output_filename = f"{audio_filename}.docx"
output_path = os.path.join("data", output_filename)
doc.save(output_path)
print(f"Results saved to '{output_path}'.")
pbar.update(1)

# İlerleme çubuğunu tamamla
pbar.close()


Process Step:   0%|          | 0/10 [00:00<?, ?step/s]

Device: cpu, Compute Type: int8
Loading WhisperX model...
No language specified, language will be first be detected for each audio file (increases inference time).


C:\Users\musta\AppData\Local\Programs\Python\Python310\lib\inspect.py:869: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  if ismodule(module) and hasattr(module, '__file__'):
Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint c:\Users\musta\Documents\Python Projects\tangible\venv_whisperx\lib\site-packages\whisperx\assets\pytorch_model.bin`
Process Step:  20%|██        | 2/10 [00:34<02:18, 17.29s/step]

Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.5.1+cpu. Bad things might happen unless you revert torch to 1.x.
WhisperX model loaded.
Audio file: data\Görüşme10 Selma Zengin.m4a
Transcribing audio file...


Process Step:  40%|████      | 4/10 [1:49:59<3:13:47, 1937.93s/step]

...

c:\Users\musta\Documents\Python Projects\tangible\venv_whisperx\lib\site-packages\pyannote\audio\models\blocks\pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\ReduceOps.cpp:1823.)
  std = sequences.std(dim=-1, correction=1)
Process Step:  80%|████████  | 8/10 [3:29:08<1:08:15, 2047.69s/step]

Speaker diarization completed.
Speaker information is being merged with the transcription...


Process Step:  90%|█████████ | 9/10 [3:29:09<23:55, 1435.59s/step]  

Merging completed.
Saving results to the Word document...
Results saved to 'data\Görüşme10 Selma Zengin.docx'.


Process Step: 100%|██████████| 10/10 [3:29:09<00:00, 1254.93s/step]
